# Manejo y registro de modelos con MLFlow

## Importar librerías

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pickle
import scipy
import os
from datetime import datetime
import xgboost
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import mlflow
import warnings

warnings.filterwarnings("ignore")

In [ ]:
mlflow.set_tracking_uri()
mlflow.set_experiment()

### Functions

In [ ]:
def load_pickle(filename:str):
    """this functions loads a pickle file
    Args:
        filename (str): path to the pickle file
    Returns:
        object: the object contained in the pickle file"""
    with open(filename, "rb") as f_in:
        return pickle.load(f_in)

In [ ]:
X_train, y_train = load_pickle(os.path.join("data/data_processed", "train.pkl"))
X_val, y_val = load_pickle(os.path.join("data/data_processed", "valid.pkl"))
X_test, y_test = load_pickle(os.path.join("data/data_processed", "test.pkl"))

In [ ]:
def run_lasso_regression(X_train: scipy.sparse._csr.csr_matrix, y_train: scipy.sparse._csr.csr_matrix, 
                         X_val: scipy.sparse._csr.csr_matrix, y_val: scipy.sparse._csr.csr_matrix, 
                         X_test: scipy.sparse._csr.csr_matrix, y_test: scipy.sparse._csr.csr_matrix):
    """
    
    Perform Lasso regression using the provided training, validation, and test data, 
    remember we are using as a backend store uri mlflow.db
    copy this in the terminal mlflow ui --backend-store-uri sqlite:///mlflow.db
    
    Parameters:
    - X_train: Training feature data
    - y_train: Training target data
    - X_val: Validation feature data
    - y_val: Validation target data
    - X_test: Test feature data
    - y_test: Test target data

    Returns:
    - Trained Lasso regression model
    """

    


In [ ]:
run_lasso_regression(X_train, y_train, X_val, y_val, X_test, y_test)

Validación de hiperparametros con optimización del rmse con Hyperot. 

In [ ]:
import xgboost as xgb
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.pyll import scope

In [ ]:
train = xgb.DMatrix(X_train, label=y_train)
valid = xgb.DMatrix(X_val, label=y_val)
test = xgb.DMatrix(X_test, label=y_test)


In [ ]:
def objective(params: dict):
    """
    Optimize an XGBoost model using hyperopt and MLflow.
    Parameters:
    - params (dict): Dictionary containing XGBoost hyperparameters.
    Returns:
    - dict: Dictionary containing loss and status for the hyperparameter optimization.
    """
    

    

In [ ]:
search_space = {
    'max_depth': scope.int(hp.quniform('max_depth', 4, 100, 1)),
    'learning_rate': hp.loguniform('learning_rate', -3, 0),
    'reg_alpha': hp.loguniform('reg_alpha', -5, -1),
    'reg_lambda': hp.loguniform('reg_lambda', -6, -1),
    'min_child_weight': hp.loguniform('min_child_weight', -1, 3),
    'objective': 'reg:linear',
    'seed': 42
}

best_result = fmin(
    fn=objective,
    space=search_space,
    algo=tpe.suggest,
    max_evals=50,
    trials=Trials()
)

Vamos a la UI y simulemos que algún cientifico de datos o miembro del equipo decide registrar ciertos modelos en "staging", y más adelante nosotros como ingenieros de ML tomamos la decisión de cual modelo registrar para producción bajo las métricas de performance obtenidas y las pruebas. Es muy importante hacer las pruebas de validación antes de considerar un modelo en producción. Entre ellas se encuentran las siguientes:

#### 1. Pruebas de funcionalidad:

* Funcionalidad del Modelo: Asegurarse de que el modelo funcione correctamente en el entorno de producción con los datos reales (Se puede simular con cierta parte de la data o toda idealmente).
* Integración del Modelo: Verificar que el modelo integrado en la infraestructura de producción responda y se comunique adecuadamente con otros componentes del sistema (partes del Pipeline).

####  2. Pruebas de rendimiento:
* Rendimiento del Modelo: Evaluar el rendimiento del modelo en términos de velocidad de inferencia, uso de recursos (CPU, memoria), y latencia para garantizar que cumple con los requisitos de producción. (El tiempo es un factor muy importante en la generación de soluciones de negocio)
* Escala y Carga: Probar el modelo bajo cargas esperadas para asegurar que el sistema es escalable y puede manejar la demanda prevista. (Orar para que el Kernel no muera con la carga, de lo contrario mirar formas de optimizar el código, hacer carga de datos por chuncks, paralelizar procesos, etc)

#### 3. Pruebas de robustez:
* Robustez de estrés: Evaluar el comportamiento del modelo en condiciones adversas, como datos de entrada no esperados, valores faltantes, valores atípicos, etc. (Esto es muy importante, ya que en la vida real los datos no son perfectos, y es muy probable que el modelo se caiga si no se tiene en cuenta este factor por lo que generar pruebas unitarias para este tipo de casos es muy importante). 


### Making predictions from MLflow artifacts

Predict on Pandas DataFrame: 

In [ ]:
logged_model = 
# Load model as a PyFuncModel.
loaded_model = 

In [ ]:
# get info from the loaded model
loaded_model

In [ ]:
xgboost_model = 

## Model registry and state transitions

<p align="center">
  <img src="https://pbs.twimg.com/media/EoOBoWyWEAAA8In.jpg" width="400" style="display: block; margin: auto;">
</p>


* Tener en cuenta el tiempo de ejecución.

* Métricas obtenidas.

* Tamaño del modelo.

* Las pruebas que anteriormente mencionamos. 


In [ ]:
from mlflow.tracking import MlflowClient


In [ ]:
MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [ ]:

# Extrae los IDs únicos de los experimentos


### Promote a model

In [ ]:
run_id = "a90d6cc190014a21ba5a0dc4de13f94e"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri, "regressor_model")

Cuando vayas a cambiar el staging de un modelo a otro estado, podemos también añadir descripciones: 

### Testing a register model

De esta forma se deben hacer las pruebas con el pipeline de producción para asegurar que el modelo se comporta como se espera. Además de validar la infraestructura y las pruebas.

In [ ]:
# OJO: Es importante que recuerdes los formatos permitidos por mlflow load_model, como dataframes, numpy arrays, etc.
def testint_model_from_mlflow(model_name: str, stage:str, X_test: xgboost.core.DMatrix, Y_test: np.ndarray):
    """this function tests a model from mlflow
    Args:
        model_name (str): name of the model
        stage (str): stage of the model
        X_test (scipy.sparse._csr.csr_matrix): test data
        Y_test (scipy.sparse._csr.csr_matrix): test target
    Returns:
        float: rmse of the model
    
    """
    model_uri = f"models:/{model_name}/{stage}"
    model = mlflow.pyfunc.load_model(model_uri)
    y_pred = model.predict(X_test)
    rmse = mean_squared_error(Y_test, y_pred, squared=False)
    return {"rmse": rmse}

In [ ]:
%time
testint_model_from_mlflow(model_name= "regressor_model", stage="Production", X_test=test, Y_test=test.get_label())


#### Ventajas de MLflow:

* Gestión de Ciclo de Vida: Facilita el seguimiento de experimentos, versionado de modelos y reproducción de resultados.
* Interoperabilidad: Es compatible con múltiples frameworks de aprendizaje automático y se integra fácilmente en flujos de trabajo existentes.
* Abierto y Modular: Ofrece una arquitectura modular que permite la flexibilidad y personalización.
* Trazabilidad y Reproducibilidad: Registra métricas, parámetros y artefactos para reproducir modelos y resultados.
* Comunidad Activa: Amplia comunidad de usuarios y contribuciones continuas.


#### Desventajas de MLflow:
* Complejidad para Grandes Volúmenes de Datos: Puede enfrentar dificultades al manejar grandes volúmenes de datos o flujos de trabajo muy complejos.
* Curva de Aprendizaje: Requiere tiempo para familiarizarse con todas sus funcionalidades y componentes.
Limitaciones en Algunas Funcionalidades: Algunas funcionalidades pueden no ser tan avanzadas o flexibles como en otras herramientas especializadas.


### Alternativas a MLflow:
* TensorBoard: Enfoque específico para TensorFlow, útil para visualizar gráficamente métricas, grafos de modelos y más.
* DVC (Data Version Control): Se enfoca en versionado de datos y modelos, y gestión de experimentos.
* Comet.ml: Ofrece seguimiento de experimentos, colaboración y visualización de manera similar a MLflow.
* Weights & Biases: Ofrece seguimiento de experimentos, colaboración y visualización de manera similar a MLflow.


#### Conclusiones

* Podemos trackear metadata

* Registrar modelos

* Obtener los requirimientos del ambiente de desarrollo donde fue entrenado los modelos

* Podemos hacer un seguimiento de los modelos y compararlos de forma fácil y amigable con la interfaz de MLflow y en código

* Hacer transiciones de estados de los modelos

* Añadir anotaciones o descripciones 

So, ya sabes incluír el registro de modelos en nuestro pipeline de ML, ahora vamos a aprender de workflows y tasks. ¡Te espero en la próxima clase! :) 